# 1 量化 

## 1.1 MinMax Quantization（INT8 基础）

**目标：** 了解最基础的 MinMax 量化、symmetric vs asymmetric、percentile clipping。

### 1.1.1 对称量化（symmetric）

给定实值张量 $x$，INT8 对称量化范围为 $[-127, 127]$：

- 量化步长：

$$
s = \frac{\max(|x|)}{Q_{\max}}, \quad Q_{\max} = 127
$$

- 量化：

$$
q = \mathrm{round}\left( \frac{x}{s} \right)
$$

- 反量化：

$$
\hat{x} = s q
$$

### 1.1.2 非对称量化（asymmetric）

常用于 activation：

$$
s = \frac{x_{\max} - x_{\min}}{Q_{\max} - Q_{\min}}, \quad
z = Q_{\min} - \frac{x_{\min}}{s}
$$

$$
q = \mathrm{round}\left( \frac{x}{s} + z \right), \quad
\hat{x} = s (q - z)
$$

### 1.1.3 Percentile clipping

避免 outlier 影响：

- 使用 $[p, 1-p]$ 分位数替代全局 min/max，常见选择：
  - CNN: p ≈ 0.001
  - Transformer: p ≈ 0.01

### 文献

- Jacob et al., *Quantization and Training of Neural Networks for Efficient Integer-Arithmetic-Only Inference*, CVPR 2018.  
- PyTorch 量化实践教程（Quantization in Practice）  
  https://pytorch.org/blog/quantization-in-practice/


In [1]:
import torch

def minmax_symmetric_quant(x, num_bits=8, percentile=None):
    """对称 MinMax 量化，支持 percentile clipping。"""
    x = x.detach()
    if percentile is not None:
        lo = torch.quantile(x, (1 - percentile) / 2)
        hi = torch.quantile(x, 1 - (1 - percentile) / 2)
        max_val = torch.max(lo.abs(), hi.abs())
    else:
        max_val = x.abs().max()

    qmax = 2 ** (num_bits - 1) - 1  # INT8: 127
    scale = max_val / (qmax + 1e-8)
    scale = torch.clamp(scale, min=1e-8)

    q = torch.round(x / scale).clamp(-qmax, qmax)
    x_hat = q * scale
    return q, x_hat, scale



x = torch.randn(4096) * 3.0
for p in [None, 0.99, 0.999]:
    q, x_hat, s = minmax_symmetric_quant(x, num_bits=8, percentile=p)
    err = torch.norm(x - x_hat) / torch.norm(x)
    print(f"percentile={p}, scale={s.item():.4f}, rel L2 error={err.item():.4e}")

percentile=None, scale=0.0927, rel L2 error=8.8467e-03
percentile=0.99, scale=0.0605, rel L2 error=4.6716e-02
percentile=0.999, scale=0.0766, rel L2 error=1.4280e-02


In [2]:
# 极大tensor([1, 4096, 4096])上算quantile 会非常慢，原因$O(N logN)$ 排序。快速算法是方法是histogram calibration
def histogram_percentile_clipping(x, percentile=0.999, num_bins=2048):
    """
    Fast percentile clipping using histogram instead of sorting.
    This is O(N) instead of O(N log N) for torch.quantile.
    
    Args:
        x: Input tensor (any shape)
        percentile: Target percentile (e.g., 0.999 = 99.9%)
        num_bins: Number of histogram bins (more bins = more precision)
    
    Returns:
        threshold: The value at the given percentile
    """
    x_flat = x.view(-1).abs()  # Flatten and take absolute values
    
    # Get min and max for histogram range
    x_min = x_flat.min()
    x_max = x_flat.max()
    
    # Handle edge case: all values are the same
    if x_max == x_min:
        return x_max
    
    # Create histogram
    hist = torch.histc(x_flat, bins=num_bins, min=x_min.item(), max=x_max.item())
    
    # Calculate cumulative sum
    cumsum = torch.cumsum(hist, dim=0)
    total = cumsum[-1]
    
    # Find the bin where cumulative sum exceeds percentile * total
    target_count = total * percentile
    bin_idx = torch.searchsorted(cumsum, target_count)
    
    # Convert bin index to actual value
    # bin_idx corresponds to the right edge of that bin
    bin_width = (x_max - x_min) / num_bins
    threshold = x_min + (bin_idx + 1) * bin_width
    
    return threshold

def minmax_symmetric_quant_fast(x, num_bits=8, percentile=None, num_bins=2048):
    """
    Optimized symmetric MinMax quantization with fast histogram-based percentile clipping.
    
    Args:
        x: Input tensor
        num_bits: Quantization bit width (default: 8)
        percentile: If provided, clip to this percentile (e.g., 0.999)
        num_bins: Number of histogram bins for percentile calculation
    
    Returns:
        q: Quantized tensor (integer)
        x_hat: Dequantized tensor (float)
        scale: Scale factor used
    """
    x = x.detach()
    
    if percentile is not None:
        # Use histogram-based percentile clipping (fast!)
        max_val = histogram_percentile_clipping(x, percentile, num_bins)
    else:
        # Use simple max (fastest)
        max_val = x.abs().max()
    
    qmax = 2 ** (num_bits - 1) - 1  # INT8: 127
    scale = max_val / (qmax + 1e-8)
    scale = torch.clamp(scale, min=1e-8)
    
    q = torch.round(x / scale).clamp(-qmax, qmax)
    x_hat = q * scale
    return q, x_hat, scale


import time
# Create a large tensor
x = torch.randn(10000, 1024) * 3.0  # ~10M elements

print("Testing on tensor with", x.numel(), "elements")
print("=" * 60)

# Method 1: Original torch.quantile (slow)
print("\n1. Original torch.quantile method:")
start = time.time()
lo = torch.quantile(x, 0.0005)
hi = torch.quantile(x, 0.9995)
max_val_slow = torch.max(lo.abs(), hi.abs())
time_slow = time.time() - start
print(f"   Time: {time_slow:.4f}s")
print(f"   Threshold: {max_val_slow.item():.4f}")

# Method 2: Histogram-based (fast)
print("\n2. Histogram-based method:")
start = time.time()
max_val_fast = histogram_percentile_clipping(x, percentile=0.999, num_bins=2048)
time_fast = time.time() - start
print(f"   Time: {time_fast:.4f}s")
print(f"   Threshold: {max_val_fast.item():.4f}")
print(f"   Speedup: {time_slow/time_fast:.1f}x faster")

# Method 3: Full quantization with histogram
print("\n3. Full quantization (histogram-based percentile):")
for p in [None, 0.99, 0.999, 0.9999]:
    start = time.time()
    q, x_hat, s = minmax_symmetric_quant_fast(x, num_bits=8, percentile=p, num_bins=2048)
    elapsed = time.time() - start
    err = torch.norm(x - x_hat) / torch.norm(x)
    print(f"   percentile={p}, scale={s.item():.4f}, "
            f"L2_err={err.item():.4e}, time={elapsed:.4f}s")    

Testing on tensor with 10240000 elements

1. Original torch.quantile method:
   Time: 2.3569s
   Threshold: 9.8730

2. Histogram-based method:
   Time: 0.0123s
   Threshold: 9.8768
   Speedup: 192.0x faster

3. Full quantization (histogram-based percentile):
   percentile=None, scale=0.1336, L2_err=1.2865e-02, time=0.0122s
   percentile=0.99, scale=0.0608, L2_err=4.3415e-02, time=0.0239s
   percentile=0.999, scale=0.0778, L2_err=1.3800e-02, time=0.0186s
   percentile=0.9999, scale=0.0920, L2_err=9.4657e-03, time=0.0188s


## 1.2 KL Divergence Calibration（TensorRT 风格熵校准）

**目标：** 用 KL 散度寻找最优 clipping 阈值 $\alpha$。

- 真实分布 $P$：FP32 activation 的直方图  
- 量化重建分布 $Q(\alpha)$：clip+量化+反量化后重新统计的直方图  
- 目标：

$$
\alpha^* = \arg\min_{\alpha} D_{\text{KL}}(P \Vert Q(\alpha))
$$

其中：

$$
D_{\text{KL}}(P\Vert Q) = \sum_i P(i)\log\frac{P(i)}{Q(i)}
$$

### 1.2.1 实际工程

- TensorRT / OpenVINO 等框架都支持基于 KL 的熵校准（entropy calibration）。
- 通常：
  - 先用 histogram 估计 P；
  - 枚举多个 $\alpha$；
  - 选择 KL 最小的阈值。

### 1.2.2 文献

- Lei Mao：*Cross Entropy, KL Divergence, and Maximum Likelihood Estimation*  
  https://leimao.github.io/blog/Cross-Entropy-KL-Divergence-MLE/
- NVIDIA TensorRT Developer Guide 中关于 INT8 calibration 的章节。

In [3]:
import torch

def kl_divergence(p, q, eps=1e-8):
    p = p + eps
    q = q + eps
    return torch.sum(p * torch.log(p / q))

def kl_calibration(samples, num_bins=2048, num_candidates=80, bit=8):
    """KL calibration with consistent histogram ranges."""
    samples = samples.detach().view(-1)
    abs_samples = samples.abs()
    
    # Reference histogram range (use max of original samples)
    hist_max = abs_samples.max()
    
    # 1. 高精度直方图 P (original distribution)
    hist = torch.histc(abs_samples, bins=num_bins, min=0, max=hist_max)
    P = hist / hist.sum()
    
    # 2. 候选 alpha 使用分位数
    qs = torch.linspace(0.90, 0.9999, num_candidates)
    alphas = torch.quantile(abs_samples, qs)
    
    Q_int = 2 ** (bit - 1) - 1  # 对称 INT8
    
    best_kl = None
    best_alpha = None
    
    for alpha in alphas:
        # 3. clip + 量化 + 反量化
        clipped = torch.clamp(samples, -alpha, alpha)
        scale = alpha / (Q_int + 1e-8)
        q = torch.round(clipped / scale).clamp(-Q_int, Q_int)
        deq = q * scale
        
        # 4. 计算重建直方图 Q(α) - USE SAME RANGE AS P
        hist_q = torch.histc(deq.abs(), bins=num_bins, min=0, max=hist_max)
        Q = hist_q / hist_q.sum()
        
        kl = kl_divergence(P, Q)
        if best_kl is None or kl < best_kl:
            best_kl = kl
            best_alpha = alpha
    
    return best_alpha, best_kl


x = torch.randn(200000) * 2.5
alpha, kl = kl_calibration(x)
print("Best alpha:", alpha.item(), "KL:", kl.item())

Best alpha: 4.141835689544678 KL: 9.38576889038086


## 1.3 SmoothQuant（大模型 INT8）

### 1.3.1 数学原理
SmoohQuant 解决的是大模型量化中一个非常棘手的问题：激活值（Activation）中的异常值（Outliers）难以量化。

简单来说，大模型（如 GPT、LLaMA）的 Activation 通常非常“难搞”，而 Weight 相对“平滑”。SmoothQuant 的核心思想就是把 Activation 的量化难度转移一部分给 Weight，让两者都变得“平滑”，从而都能轻松用 INT8 量化。

1. 问题的根源：Activation Outliers
在 LLM 中，某些特定的通道（Channel）的激活值会特别大（可能是其他通道的 100 倍）。
如果你为了照顾这几个大值，把量化范围（Scale）设得很大，那么其他 99% 的小数值都会被压缩到 0 附近，导致精度归零。
如果你把范围设小，这几个大值就会被截断（Clipping），严重破坏模型逻辑。
与此同时，权重（Weight） 的分布通常很均匀，量化起来很容易。

2. 核心思想：数学变换
SmoothQuant 引入了一个平滑系数 $s$，试图把 Activation 的大值“压下去”，同时把 Weight 对应维度的值“放大上来”。
对于矩阵乘法 $Y = X W$：
$X$ 是输入激活值（很难量化，有尖峰）
$W$ 是权重（容易量化，很平坦）
我们在中间插入一个除法和一个乘法（数学上等于乘 1，不改变结果）：
$$ Y = (X \cdot \text{diag}(s)^{-1}) \cdot (\text{diag}(s) \cdot W) $$
这就变成了两个新变量：
新的激活值 $X' = X / s$
新的权重 $W' = W \cdot s$

3. 怎么选这个 $s$？
这正是 SmoothQuant 的精髓。它定义了一个超参数 $\alpha$ 来控制“搬运”多少难度。
$$ s_j = \frac{\max(|X_j|)^\alpha}{\max(|W_j|)^{1-\alpha}} $$
其中 $j$ 代表第 $j$ 个输入通道（Input Channel）。
如果 $\alpha = 1$：$s$ 完全由 Activation 的最大值决定。这意味着我们把 Activation 的所有异常值都除掉了，$X'$ 变得极度平滑（所有通道最大值都变成 1）。但是，$W'$ 会变得波动巨大（某些行被放大了 100 倍），导致 Weight 变得很难量化。
如果 $\alpha = 0$：$s$ 完全由 Weight 决定，相当于啥也没做，保持原样。
SmoothQuant 的选择（$\alpha = 0.5$）：折中！
让 $s \approx \sqrt{\max(|X|)}$。
这样 $X$ 被除以了自己的平方根，尖峰变小了。
$W$ 被乘以了 $X$ 的平方根，波动变大了，但没那么离谱。
结果： $X'$ 和 $W'$ 都处于一种“比较好量化”的状态，都能用 INT8 量化。

4. 图解示例

假设某个通道：

Activation $X$: [100, 0.1, 0.2] (极大值 100，很难量化)

Weight $W$: [0.5, 0.5, 0.5] (很均匀)

我们选 $s = 10$ ($\alpha=0.5$ 附近)：

新 $X'$ = [10, 0.01, 0.02] (最大值变成了 10，容易量化多了)
新 $W'$ = [5, 5, 5] (变成了 5，虽然变大了，但还是均匀的)

结论： 通过这个简单的数学变换，我们消除了 Activation 中的恐怖尖峰，代价仅仅是让 Weight 稍微难一点点，实现了整体最优的 INT8 量化。

数学：

对线性层 $y = XW$：

- 定义 per-channel 因子 $s_c$
- 变换：$X' = X D^{-\alpha},\ W' = D^{\alpha} W$
- 使得 X'、W' 都更容易被 INT8 量化。

### 1.3.2 文献

- *SmoothQuant: Accurate and Efficient Post-Training Quantization for Large Language Models*, arXiv:2211.10438.

In [4]:
import torch

def smoothquant_linear(X, W, alpha=0.5, eps=1e-5):
    """
    按输入通道做 SmoothQuant 缩放。
    假设 W 为 nn.Linear.weight, 形状为 [out_features, in_features]
    X 形状为 [batch, seq, in_features]
    """
    # 1. 计算 Activation 的逐通道最大值 (沿着 Batch 和 Seq 维度)
    # X: [B, L, D_in] -> act_max: [D_in]
    act_max = X.abs().amax(dim=(0, 1))
    
    # 2. 计算 Weight 的逐通道最大值 (沿着 Output Channel 维度，即 dim=0)
    # W: [D_out, D_in] -> wt_max: [D_in]
    # 注意：这里必须是 dim=0，因为我们要看每一列(每个输入通道)的最大值
    wt_max = W.abs().amax(dim=0) 

    # 3. 计算平滑系数 s: [D_in]
    s = (act_max ** alpha) / (wt_max ** (1 - alpha) + eps)
    s = torch.clamp(s, min=eps)

    # 4. 应用缩放
    # X: [B, L, D_in] / [D_in] -> 广播正确
    X_sq = X / s
    
    # W: [D_out, D_in] * [D_in] -> PyTorch 默认广播最后一维，正确
    W_sq = W * s
    
    return X_sq, W_sq

X = torch.randn(4, 128, 512)
W = torch.randn(2048, 512)
X2, W2 = smoothquant_linear(X, W)

# original
Y_orig = X @ W.T
Y_new = X2 @ W2.T

print("Max diff:", (Y_orig - Y_new).abs().max().item())

Max diff: 3.0517578125e-05


## 1.4 GPTQ（LLM 低比特 PTQ）

GPTQ 使用近似 Hessian（通常对角或 block 对角）衡量每个权重的重要性，在给定 bit 下求解近似最优的权重量化方案。

### 1.4.1 数学基础
GPTQ = “Hessian 加持的权重量化 PTQ”：

- 只做 weight-only quantization（激活保留在 FP16/BF16），主要作用在 Transformer 里的 Linear / Conv1D（QKV 投影、MLP 等）

- 典型配置：3–4 bit 权重 + 每 group（例如 128 个通道）一个 scale，可以把 LLM 压到 4bit 还能保持不错精度
- 是 post-training、one-shot：只用一小段校准数据，不需要再训。

关键点：不是 naive round-to-nearest，而是用一层的 输入 Hessian（二阶信息）来度量「哪些方向的误差更要命」。
有点像「Optimal Brain Surgeon/Optimal Brain Quantization」在 LLM 上的工程化版本：**通过 Hessian 权重的二阶近似，把量化误差往“不重要的权重方向”推**。

做法：对单层线性做二阶近似
考虑一层线性层（忽略 bias）：
$$Y=XW$$
- $X \in \mathbb{R}^{n \times d_\text{in}}$： 校准数据上的输入（多 batch 拼接后的 activation）
- $W \in \mathbb{R}^{d_\text{in} \times d_\text{out}}$：原始全精度权重。
- 我们要找量化后的 $\tilde W$，满足 bit 限制（比如 4bit）。
GPTQ 把「输出重建误差」近似为：
$$\min_{\tilde W \in \mathcal{Q}} \| XW - X\tilde W \|_F^2$$
做二阶展开，可写成对每一列$w_j$的二次型：
$$L \approx (w_j - \tilde w_j)^\top H (w_j - \tilde w_j)$$
其中$H \approx 2 X^\top X + \lambda I$是输入的 Hessian 近似（𝜆是一个小的 damping 项）

核心思想：
- 不是简单最小化 $\|w_j - \tilde w_j\|_2^2$, 而是最小化带权距离$(w_j - \tilde w_j)^\top H (w_j - \tilde w_j)$
- 如果某方向在$H$里权重大，说明改动这个方向导致输出变化很大 → 量化时要格外小心；反之，可以更“随便”地压缩。

### 1.4.3 算法
对某一层（例如一个 `nn.Linear`）：

1. 收集校准激活 X
    - 用一小段代表性的文本跑 full-precision 模型。
    - 在该层输入处挂 forward hook，收集若干 batch 的 activations。

2. 构建 Hessian 近似 & 其逆/Cholesky
$$
H = 2X^T X / N + \lambda I, \quad H^{-1} = \operatorname{inv}(H)
$$

真实实现里会用 Cholesky 分解 $H^{-1} = LL^T$ 来保证数值稳定并加速矩阵操作。

3. 按 block / group 处理权重列（out_features）
把 $W$ 的列（输出通道）分成若干 block（大小 B，比如 128），每个 block 内逐列处理：

- 对 block 中的第 $j$ 列 $w_j$：

     - 使用 k-bit 均匀对称量化 + group scale 对 $w_j$ 量化得到 $q_j$：
   $$
   q_j = \operatorname{round}\left(\frac{w_j}{s}\right) \cdot s
   $$

     - 计算量化误差 $e_j = w_j - q_j$。

     - 用 $H^{-1}$ 把这次误差对未来列的影响“预先补偿”掉：
   > 直觉是：既然第 j 列损失了一点“重要方向”的信息，就在后面的列上反向加一点回来。这一块就是 GPTQ 的「error propagation」。

4. 只把 quantized 权重保存下来
    - 激活、KV Cache 等保持 FP16/BF16。
    - 推理时使用低比特权重 + 对应的 scale/zero-point / group-wise 元数据

### 1.4.3 常用配置
在公开的 GPTQ 模型 / 文档里，常用配置：

- 权重位宽：wbits=4（有时 3 或 2）。

- group size：groupsize=128 或 64（多少个 channel 共享一个量化 scale）。

- 只量化 language model core 的线性层，其他部分（embedding、ln、vision tower 等）保持 FP16/BF16。

- 激活保持 FP16/BF16，有些库会额外支持 activation INT8，但经典 GPTQ 是 weight-only。 


实际工程里最常用的 Python 调用：AutoGPTQ / HF GPTQConfig

如果你只是想在 GM/实验环境里 快速把一个 LLM 量化成 GPTQ 权重并跑起来，通常不会自己从零写，而是用工具库，例如 [AutoGPTQ](https://pypi.org/project/auto-gptq) 或 transformers 中的 GPTQ 支持. example

```python
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
from transformers import AutoTokenizer

pretrained_model = "facebook/opt-125m"
quantized_dir = "opt-125m-gptq-4bit"

# 1) 配置 GPTQ（4bit, group-size 128）
quantize_config = BaseQuantizeConfig(
    bits=4,                 # 3/4bit 权重
    group_size=128,         # 每 128 channel 一个 scale
    damp_percent=0.01,      # Hessian damping
    desc_act=False          # 是否排序激活（有些实现会用）
)

# 2) 加载全精度模型，并进行 GPTQ 量化
model = AutoGPTQForCausalLM.from_pretrained(
    pretrained_model,
    quantize_config=quantize_config,
    device_map="auto",
)

# 3) 使用一小段校准数据跑 GPTQ（库内部会做 collect X + Hessian + 误差补偿）
tokenizer = AutoTokenizer.from_pretrained(pretrained_model, use_fast=True)
calib_texts = [
    "Quantization is a powerful technique for deploying large language models efficiently.",
    "GPTQ is a Hessian-aware post-training quantization algorithm."
]
calib_examples = [tokenizer(t, return_tensors="pt") for t in calib_texts]

model.quantize(calib_dataset=calib_examples)

# 4) 保存量化后的权重
model.save_quantized(quantized_dir)

# 5) 再次加载量化模型进行推理
quantized_model = AutoGPTQForCausalLM.from_quantized(
    quantized_dir,
    device_map="auto",
)
```

### 1.4.4 文献

- Frantar et al., *GPTQ: Accurate Post-Training Quantization for Generative Pre-trained Transformers*, arXiv:2210.17323  
- 官方实现：https://github.com/IST-DASLab/gptq

### 1.4.5 e2e drive model注意点
1. GPTQ 是 weight-only quantization → 对 transformer E2E pipeline 降低不了足够的 latency
GPTQ 优化点：

- 只压 weights（3–4bit）

- 激活仍然是 FP16/BF16

- KV cache/attention 仍然是 FP16/BF16

- compute bound 大头仍然在 activation matmul / attention，不是权重本身

👉 对 LLAMA 这种 decoder LLM 可以省模型大小
但对 AD 的 compute-bound Vision/Transformer latency 几乎不降。

所以 AD 场景用 GPTQ：收益太小，不值得。

2. 车端的模型不像 LLM 是“weights dominated”；AD 模型是 activation dominated

视觉部分（例如 FasterViT）和 BEV encoder 类模块：

- activation tensor 大且 dynamic

- conv/attention FLOPs 主导 latency

- weight size 本来就不大（几十~几百 MB）

- GPTQ 只压 weights，对算力占比不到 15% （有待验证）的部分做优化，对latency没有太大影响

### GPTQ 二阶近似推导

#### 1. 问题设定：单层线性与量化目标

考虑一层线性层（忽略 bias）：

$Y = X W$

其中：

- $X \in \mathbb{R}^{N \times d_{\text{in}}}$：校准数据上的输入（所有 batch 拼在一起）
- $W \in \mathbb{R}^{d_{\text{in}} \times d_{\text{out}}}$：原始全精度权重
- $\tilde W$：量化后的权重（满足 k-bit 约束）

我们希望量化后输出尽量接近原输出，整体目标可以写成：

$\min_{\tilde W \in \mathcal{Q}} \|XW - X\tilde W\|_F^2$

对第 $j$ 个输出通道（即第 $j$ 列权重）单独来看：

- 原始列：$w_j \in \mathbb{R}^{d_{\text{in}}}$
- 量化列：$\tilde w_j \in \mathbb{R}^{d_{\text{in}}}$

该列对应的输出误差为：

$L = \|X w_j - X \tilde w_j\|_2^2$

---

#### 2. 写成权重差的形式：得到二次型

定义该列的权重误差：

$\delta w_j = w_j - \tilde w_j$

则：

$L = \|X(w_j - \tilde w_j)\|_2^2 = \|X \delta w_j\|_2^2$

展开：

$L = (X \delta w_j)^\top (X \delta w_j) = \delta w_j^\top (X^\top X)\, \delta w_j$

关键结论：

- 权重误差 $\delta w_j$ 经由输入矩阵 $X$ 扩散到输出空间  
- 误差的“严重程度”由 $X^\top X$ 决定  
- 这是一个标准的二次型（quadratic form）

---

#### 3. 与二阶泰勒展开（Hessian 近似）的联系

在 PTQ 中，模型已训练完毕，满足：

$\nabla \mathcal{L}(w_j) \approx 0$

对 $\mathcal{L}(w_j)$ 做二阶泰勒展开：

$\mathcal{L}(w_j + \delta w_j)
\approx
\mathcal{L}(w_j)
+
\nabla \mathcal{L}(w_j)^\top \delta w_j
+
\frac{1}{2}\delta w_j^\top H \, \delta w_j$

由于一阶梯度为零，损失变化主要来自二阶项：

$\Delta \mathcal{L}
\approx
\frac{1}{2}\delta w_j^\top H \, \delta w_j$

GPTQ 采用输入统计的 Hessian 近似：

$H \approx 2 X^\top X + \lambda I$

其中 $\lambda I$ 是 damping 项，用于提高数值稳定性。

代入得：

$L \approx \delta w_j^\top H \, \delta w_j
= (w_j - \tilde w_j)^\top H (w_j - \tilde w_j)$

这就是 GPTQ 的核心优化目标。

---

#### 4. 对比普通 L2 量化：为什么要引入 Hessian？

普通的 PTQ 等价于：

$\min_{\tilde w_j} \|w_j - \tilde w_j\|_2^2$

假设所有方向量化敏感度相同。

GPTQ 的目标是：

$\min_{\tilde w_j}
(w_j - \tilde w_j)^\top H (w_j - \tilde w_j)$

含义：

- 若 $H$ 某方向特征值大 → 对 loss 更敏感 → 该方向量化误差必须更小  
- 若特征值小 → 对 loss 不敏感 → 可以更激进压缩  

因此 GPTQ 能在相同 bit 数下降低精度损失。

---

#### 5. 小结

从 $\|XW - X\tilde W\|_F^2$ 出发：

1. 分解单列：  
   $L = \|X w_j - X \tilde w_j\|_2^2$

2. 改写为权重误差：  
   $L = \delta w_j^\top (X^\top X) \delta w_j$

3. 用二阶近似（收敛点一阶项为零）：  
   $\Delta \mathcal{L} \approx \frac{1}{2}\delta w_j^\top H \delta w_j$

4. 用 Hessian 近似 $H \approx 2X^\top X + \lambda I$：  
   $L \approx (w_j - \tilde w_j)^\top H (w_j - \tilde w_j)$

最终 GPTQ 的目标为：

$\min_{\tilde w_j \in \mathcal{Q}}
(w_j - \tilde w_j)^\top H (w_j - \tilde w_j)$

### 1.5 AWQ（Activation-aware Weight Quantization）

AWQ 是一种 **weight-only 低比特量化** 方法（典型是 W4A16），核心两件事：

1. 发现：**权重本身长得“怪不怪”不重要，重要的是这个通道在真实激活分布下的影响有多大**。  
2. 做法：在不改 kernel 形式（仍然是 group-wise weight-only quant）的前提下，通过一个 **数学等价变换**：
   - 对“重要通道”的权重放大（scale up），
   - 对应地对输入激活缩放回来（scale down），
   - 让这些通道在同一个 group 内占到更大比重，从而 **在 group-wise 量化下得到更小的量化误差**。

---

#### 1.5.1 先从 group-wise weight-only quant 的误差出发

考虑一层线性层（忽略 bias）：

- 输入：$x \in \mathbb{R}^{d_{\text{in}}}$
- 权重：$w \in \mathbb{R}^{d_{\text{in}}}$（先只看单输出通道）
- 输出：$y = w^\top x$

对权重做 **weight-only 量化**，比如对 $w$ 做统一的定点量化：

- 量化步长（scale）  
  $$
  \Delta = \frac{\max\lvert w \rvert}{2^{b-1}-1}
  $$
- 量化权重  
  $$
  Q(w) = \Delta \cdot \text{Round}\!\left(\frac{w}{\Delta}\right)
  $$

推理时用的是

$$
\hat{y} = Q(w)^\top x
$$

于是量化误差为

$$
e = \hat{y} - y = \big(Q(w) - w\big)^\top x
= \left(\Delta \cdot \text{Round}\!\left(\frac{w}{\Delta}\right) - w\right)^\top x
$$

把 **Round 误差** 单独记出来：

$$
\text{RoundErr}\!\left(\frac{w}{\Delta}\right)
= \text{Round}\!\left(\frac{w}{\Delta}\right) - \frac{w}{\Delta}
$$

从量化定义出发：
$$Q(w)
= \Delta \cdot \text{Round}\left(\frac{w}{\Delta}\right)$$

把右边拆成两项：
$$Q(w)
= \Delta \left[
\frac{w}{\Delta}
+ \left(
\text{Round}\left(\frac{w}{\Delta}\right)
- \frac{w}{\Delta}
\right)
\right]$$

整理括号：
$$Q(w)
= w + \Delta \cdot 
\left(
\text{Round}\left(\frac{w}{\Delta}\right)
- \frac{w}{\Delta}
\right)$$

我们定义的 RoundErr 正好是括号里的这一项，所以：
$$Q(w)
= w + \Delta \cdot \text{RoundErr}\left(\frac{w}{\Delta}\right)$$

于是量化误差：
$$
Q(w) - w
= \Delta \cdot \text{RoundErr}\!\left(\frac{w}{\Delta}\right)
$$

代回去：

$$
e = \Delta \cdot \text{RoundErr}\!\left(\frac{w}{\Delta}\right)^\top x
$$

如果假设 $\frac{w}{\Delta}$ 在每个量化 bin 内“比较均匀”，于是 **期望的量化绝对误差** 满足一个非常粗但重要的关系：

$$
\mathbb{E}\lvert e \rvert
\propto \Delta \cdot \mathbb{E}\lvert x \rvert
$$

- 对同一个 group 来说，$\Delta$ 被 **所有通道共享**（group-wise quant）。
- 因此，在同一个 group 内，“谁的 $x$ 越大，谁的误差贡献越大”。

这就是 AWQ 的关键观察之一：

> **在 group-wise weight-only quant 里，量化误差其实是 activation-aware 的：  
> 激活越大的通道越“脆弱”，更值得被保护。**

---

#### 1.5.2 数学等价变换：scale 权重、反向 scale 激活

我们再看一层线性层的矩阵形式（忽略 bias）：

- 输入 batch：$X \in \mathbb{R}^{n \times d_{\text{in}}}$
- 权重：$W \in \mathbb{R}^{d_{\text{in}} \times d_{\text{out}}}$
- 输出：$Y = X W$

AWQ 的核心 trick 是一个 **等价变换**：

引入一个对 output 通道的缩放向量 $s \in \mathbb{R}^{d_{\text{out}}}$，构造对角矩阵  
$S = \mathrm{diag}(s) \in \mathbb{R}^{d_{\mathrm{out}} \times d_{\mathrm{out}}}$，则有

$$
Y = X W
= X (W S) S^{-1}
$$

其中

- $\tilde{W} = W S \in \mathbb{R}^{d_{\text{in}} \times d_{\text{out}}}$：**把权重按通道放大/缩小**
- $Y_j = (X W_j) \cdot s_j^{-1}, \quad j = 1 \dots d_{\mathrm{out}}$：**把输入激活按相反因子缩放**
- $S^{-1}$ 是(它作用在输出通道维度，不是输入通道维度)：
$$\begin{array}{c} S^{-1} = 
\begin{bmatrix}
s_1^{-1} & 0 & \cdots & 0 \\
0 & s_2^{-1} & \cdots & 0 \\
\vdots & & \ddots & \vdots \\
0 & 0 & \cdots & s_{d_{\mathrm{out}}}^{-1}
\end{bmatrix} \end{array}$$


显然这是完全等价的变换，FP32 下数值理论上不变。

现在我们对 $\tilde{W}$ 做 group-wise weight-only 量化：

$$
\hat{Y}
= \tilde{X} \, Q(\tilde{W})
= X S^{-1} Q(W S)
$$

所以我们实际做的是：

- **离线**：找到合适的 $S$，对 $W S$ 做 group-wise 量化，存成定点权重。  
- **在线推理**：在 Linear 前面插一个 cheap 的 per-channel scale（或者 fuse 进 kernel）：
  $$
  \hat{Y} = (X S^{-1}) \cdot Q(W S)
  $$

AWQ 的问题就变成了：如何选 $S$ 使得

$$
\hat{Y} = X S^{-1} Q(W S) \approx Y = X W
$$

---

#### 1.5.3 目标函数：用 activation 统计选择“重要通道”

在一层里，我们希望减小量化后的输出误差：

$$
\min_{S} \; \mathbb{E}_{X}\big\|X W - X S^{-1} Q(W S)\big\|_F^2
$$

真实情况中：

- 我们只有一个 **校准集**（calibration set）$\{X^{(k)}\}_{k=1}^K$，通常是从训练/推理分布里采样若干 batch。
- AWQ 在实现上采用若干近似，使得搜索变得 **局部、分组、one-shot**，不需要反向传播。

关键 heuristic：

1. **重要通道判定**  
   对于某一层的输出通道 $j$，用校准数据计算：
   $$
   I_j = \mathbb{E}_X\lvert Y_{:, j} \rvert 
   = \mathbb{E}_X\lvert (X W)_{:, j} \rvert
   $$
   或者用 max/percentile 来度量这个通道在真实激活下的“能量”。

   $I_j$ 越大，说明这个通道在输出中贡献越大，**越脆弱**，应该在量化时被保护。

2. **只对“重要通道”提高 scale**  
   对 $I_j$ 排序，只选 top-$p\%$ 的通道（比如 1%–5%）赋予更大的缩放因子 $s_j > 1$，  
   其余通道保持 $s_j \approx 1$。

3. **在 group 内 search / greedily 优化 $s_j$**  
   在同一个 group 里，权重共享一个量化步长 $\Delta_{\text{group}}$。  
   我们希望放大重要通道 $j$ 的权重 $W_{:, j}$，同时不要把 group 的总体范围放大得太离谱，否则 $\Delta_{\text{group}}$ 变大又会伤害其他通道。  
   实现里通常在一个离散候选集合中（比如 $\{1, 2, 4, 8\}$）搜索合适的 $s_j$，用 calibration 的重建误差来评估。

从误差形式可以看出：

- 如果某个通道 $j$ 的输出激活 $Y_{:, j}$ 很大，我们希望通过 $s_j$ 让它在 group-wise 量化后更接近原始值；
- 同时，由于我们是用 $S^{-1}$ 去缩放输入激活，对于被放大的通道 $j$，输入激活实际上被 **缩小**，从而降低了前面推导中的 $\mathbb{E}\lvert x\rvert$，进一步降低量化误差。

最终使重要通道的误差变小。

因此 AWQ 的效果介于：

- 传统的 group-wise weight-only quant（误差较大但 kernel 高效）
- 和 per-channel quant（误差更小但 kernel/metadata 开销更大）

之间，**用 activation-aware 的 scale 来“接近” per-channel 的效果，同时保留 group-wise kernel 的效率**。

---

#### 1.5.4 简化版单层 AWQ 算法（伪代码）

对某一层 Linear：$Y = X W$：

1. **收集校准激活**
   - 用校准数据跑若干 batch，记录这层的输入 $X$ 与输出 $Y = X W$。
2. **计算通道重要性**
   - 每个输出通道 $j$：
     $$
     I_j = \frac{1}{N}\sum_{k=1}^N \lvert Y^{(k)}_{:, j} \rvert_{\text{mean or max}}
     $$
3. **选重要通道集合 $\mathcal{C}$**
   - 取 $I_j$ top-$p\%$ 的通道索引。
4. **在每个 group 内搜索缩放因子**
   - 假设 group 大小为 $G$（例如 128 输出通道为一组）。
   - 对每个 group $g$，在候选缩放集合（例如 $\{1,2,4,8\}$）上，对 group 内的通道 $j \in g \cap \mathcal{C}$ 进行搜索：
     - 暂时设定 $s_j$，形成 $S$。
     - 计算 $\hat{Y}^{(k)} = X^{(k)} S^{-1} Q(W S)$ 的重建误差。
     - 选择误差最小的组合（通常是 greedy/coordinate descent，而不是真正的全局搜索）。
5. **保存量化后的权重和 scale**：
   - 存储 $Q(W S)$ 和每个通道的 $s_j$（或者折叠到下游层里）。

---

#### 1.5.4 PyTorch 风格的 AWQ toy 实现示例

> 说明：下面是 **教学用简化版**，不是官方 `llm-awq` 的完整实现，只是把上面的数学思路变成可跑的代码结构，方便你在小模型上做实验。真正工程版 AWQ 支持 LLaMA 等 LLM、INT3/4、高效 CUDA kernel、模型 zoo 等，可以直接参考官方仓库。

思路：

- 写一个 `awq_quantize_linear`，对单个 `nn.Linear` 做：
  - 采样一批校准输入，前向得到输出来算通道重要性；
  - 生成 per-channel 缩放 $s_j$；
  - 构造新的量化 Linear 模块（存 int 权重 + scale）。
- 写一个简单的 `AWQLinear`，在 `forward` 里实现：
  - $x' = x \cdot S^{-1}$  
  - $y = x' @ W_{\text{int}}^\top \cdot \Delta$（group-wise）  



In [ ]:
import torch
import torch.nn as nn
from typing import Tuple

class AWQLinear(nn.Module):
    def __init__(self, W_fp32, bias_fp32=None, group_size=128, nbit=4, s=None):
        super().__init__()
        assert W_fp32.ndim == 2
        self.in_features, self.out_features = W_fp32.shape
        self.group_size = group_size
        self.nbit = nbit

        if s is None:
            s = torch.ones(self.out_features, device=W_fp32.device, dtype=W_fp32.dtype)
        self.register_buffer("s", s)

        # bias 不量化，直接拷贝保存
        if bias_fp32 is not None:
            self.register_buffer("bias", bias_fp32.clone())
        else:
            self.bias = None

        # 先按通道 scale 权重
        W_scaled = W_fp32 * self.s  # (d_in, d_out)
        W_int, group_scales = self._groupwise_quantize(W_scaled, group_size, nbit)

        self.register_buffer("W_int", W_int)
        self.register_buffer("group_scales", group_scales)

    def _groupwise_quantize(self, W, group_size, nbit):
        d_in, d_out = W.shape
        num_groups = (d_out + group_size - 1) // group_size
        W_int = torch.zeros_like(W)
        group_scales = torch.zeros(num_groups, device=W.device, dtype=W.dtype)
        qmax = 2 ** (nbit - 1) - 1

        for g in range(num_groups):
            start, end = g * group_size, min((g + 1) * group_size, d_out)
            W_group = W[:, start:end]
            maxv = W_group.abs().max()
            if maxv < 1e-8:
                scale = torch.tensor(1.0, device=W.device, dtype=W.dtype)
            else:
                scale = maxv / qmax
            group_scales[g] = scale

            W_q = torch.clamp(torch.round(W_group / scale), -qmax - 1, qmax)
            W_int[:, start:end] = W_q

        return W_int.to(torch.int8), group_scales

    def forward(self, x):
        """
        把 group_scales 和 s 还原成 per-channel 的 fp32 权重再 matmul。
        工程里会 fuse 到 kernel，这里只是演示。
        """
        d_out = self.out_features
        group_size = self.group_size
        num_groups = self.group_scales.numel()
        device = x.device

        # 展开 group_scales → 每个输出通道一个 scale
        per_channel_scale = torch.zeros(d_out, device=device, dtype=self.group_scales.dtype)
        for g in range(num_groups):
            start, end = g * group_size, min((g + 1) * group_size, d_out)
            per_channel_scale[start:end] = self.group_scales[g]

        # 反量化 + 除以 s（等价变换）
        W_int_fp = self.W_int.to(per_channel_scale.dtype)
        W_dequant = W_int_fp * per_channel_scale  # (d_in, d_out)
        W_dequant = W_dequant / self.s            # 每个输出通道除以 s_j

        y = x @ W_dequant
        if self.bias is not None:
            y = y + self.bias
        return y


@torch.no_grad()
def awq_search_scales_for_linear(
    linear,
    calib_loader,
    nsamples=128,
    group_size=128,
    nbit=4,
    top_p=0.01,
    alpha=0.5,
    device="cuda",
):
    linear = linear.to(device).eval()
    # 转置成 (d_in, d_out)
    W = linear.weight.data.t().clone().to(device)
    bias = None
    if linear.bias is not None:
        bias = linear.bias.data.clone().to(device)

    d_out = W.shape[1]

    acts_in, acts_out = [], []
    ncollected = 0
    for batch in calib_loader:
        x = batch.to(device)
        y = linear(x)
        acts_in.append(x)
        acts_out.append(y)
        ncollected += x.shape[0]
        if ncollected >= nsamples:
            break

    X = torch.cat(acts_in, dim=0)[:nsamples]
    Y = torch.cat(acts_out, dim=0)[:nsamples]

    # 通道重要性：简单用 mean |Y|
    importance = Y.abs().mean(dim=0)
    k = max(1, int(d_out * top_p))
    _, idx = torch.topk(importance, k)
    important_mask = torch.zeros(d_out, dtype=torch.bool, device=device)
    important_mask[idx] = True

    # 构造 per-channel s
    # 实际论文在给定的校准数据X上，真正去算「量化前后输出的重建误差L2」：
    # L(s)=||XW−X′Wq′​(s)|| 
    # 然后对重要通道做贪心搜索
    # 此处只是用输出大小 proxy 了“重要性”，却没有做真正的误差搜索
    s = torch.ones(d_out, device=device, dtype=W.dtype)
    ratio = (importance / (importance.mean() + 1e-6)).clamp(min=1.0)
    s[important_mask] = (ratio[important_mask] ** alpha).clamp(max=8.0)

    awq_linear = AWQLinear(
        W_fp32=W,
        bias_fp32=bias,
        group_size=group_size,
        nbit=nbit,
        s=s,
    )
    return awq_linear


# 对小 MLP 做 AWQ，并对比误差
class SmallMLP(nn.Module):
    def __init__(self, d_in=64, d_hidden=128, d_out=32):
        super().__init__()
        self.fc1 = nn.Linear(d_in, d_hidden)
        self.act = nn.GELU()
        self.fc2 = nn.Linear(d_hidden, d_out)

    def forward(self, x):
        return self.fc2(self.act(self.fc1(x)))


def build_calib_loaders(model, n_samples=512, batch_size=32, device="cuda"):
    model = model.to(device).eval()
    with torch.no_grad():
        # 原始输入校准数据 (N, 64)
        x_calib = torch.randn(n_samples, 64, device=device)
        # 通过 fc1+GELU 得到中间激活 (N, 128)
        h_calib = model.act(model.fc1(x_calib))

    def make_loader(data):
        return [data[i:i + batch_size] for i in range(0, n_samples, batch_size)]

    calib_loader_fc1 = make_loader(x_calib)  # 给 fc1 用
    calib_loader_fc2 = make_loader(h_calib)  # 给 fc2 用
    return calib_loader_fc1, calib_loader_fc2


In [65]:
def plain_w4(linear, group_size=32, device="cuda"):
    W = linear.weight.data.t().clone().to(device)
    bias = linear.bias.data.clone().to(device) if linear.bias is not None else None

    d_in, d_out = W.shape
    num_groups = (d_out + group_size - 1) // group_size
    W_int = torch.zeros_like(W)
    group_scales = torch.zeros(num_groups, device=W.device)

    qmax = 2 ** (4 - 1) - 1

    for g in range(num_groups):
        start = g * group_size
        end = min((g+1) * group_size, d_out)
        W_group = W[:, start:end]
        maxv = W_group.abs().max()
        scale = maxv / qmax if maxv > 1e-8 else 1.0
        group_scales[g] = scale
        W_q = torch.clamp(torch.round(W_group / scale), -qmax - 1, qmax)
        W_int[:, start:end] = W_q

    # 展开 scale
    per_channel_scale = torch.zeros(d_out, device=W.device)
    for g in range(num_groups):
        start = g * group_size
        end = min((g+1) * group_size, d_out)
        per_channel_scale[start:end] = group_scales[g]

    W_dequant = W_int * per_channel_scale

    # 构造一个普通 Linear
    qlinear = nn.Linear(W.shape[0], W.shape[1], bias=bias is not None).to(device)
    qlinear.weight.data = W_dequant.t().contiguous()
    if bias is not None:
        qlinear.bias.data = bias
    return qlinear


def main():
    import random
    device = "cuda" if torch.cuda.is_available() else "cpu"
    torch.manual_seed(0)

    model = SmallMLP().to(device).eval()

    calib_loader_fc1, calib_loader_fc2 = build_calib_loaders(
        model, n_samples=512, batch_size=32, device=device
    )

    x_test = torch.randn(16, 64, device=device)
    with torch.no_grad():
        y_ref = model(x_test)

    # -------------------------
    #  Plain W4（无 AWQ） baseline
    # -------------------------
    plain_fc1 = plain_w4(model.fc1, group_size=128, device=device)
    plain_fc2 = plain_w4(model.fc2, group_size=128, device=device)

    class PlainMLP(nn.Module):
        def __init__(self, q1, q2):
            super().__init__()
            self.fc1 = q1
            self.act = nn.GELU()
            self.fc2 = q2
        def forward(self, x): return self.fc2(self.act(self.fc1(x)))

    plain_model = PlainMLP(plain_fc1, plain_fc2).to(device)
    with torch.no_grad():
        y_plain = plain_model(x_test)

    plain_l2 = (y_plain - y_ref).pow(2).mean().sqrt().item()
    plain_max = (y_plain - y_ref).abs().max().item()

    # -------------------------
    #        AWQ  W4
    # -------------------------
    awq_fc1 = awq_search_scales_for_linear(
        model.fc1, calib_loader_fc1, nsamples=256,
        group_size=128, nbit=4, top_p=0.02, alpha=0.5, device=device
    )
    awq_fc2 = awq_search_scales_for_linear(
        model.fc2, calib_loader_fc2, nsamples=256,
        group_size=128, nbit=4, top_p=0.02, alpha=0.5, device=device
    )

    class QuantMLP(nn.Module):
        def __init__(self, q1, q2):
            super().__init__()
            self.fc1 = q1
            self.act = nn.GELU()
            self.fc2 = q2
        def forward(self, x): return self.fc2(self.act(self.fc1(x)))

    qmodel = QuantMLP(awq_fc1, awq_fc2).to(device)
    with torch.no_grad():
        y_q = qmodel(x_test)

    awq_l2 = (y_q - y_ref).pow(2).mean().sqrt().item()
    awq_max = (y_q - y_ref).abs().max().item()

    # -------------------------
    # Print results
    # -------------------------
    print("==== W4 Quantization Comparison ====")
    print(f"Plain W4 L2 error : {plain_l2:.6f}")
    print(f"Plain W4 Max error: {plain_max:.6f}")
    print("---")
    print(f"AWQ  W4 L2 error  : {awq_l2:.6f}")
    print(f"AWQ  W4 Max error : {awq_max:.6f}")


main()

==== W4 Quantization Comparison ====
Plain W4 L2 error : 0.021071
Plain W4 Max error: 0.078114
---
AWQ  W4 L2 error  : 0.021870
AWQ  W4 Max error : 0.064620


不是一个好例子。
- SmallMLP weights 随机生成， 
- 没有 learned structure，
- 通道 importance（mean |Y|）分布几乎是噪声，根本不存在什么“极端重要的通道”。 
- 实现也不是完全官方算法
- 同时activate-aware应该是针对于LLM。但是至少AWQ ≈ Plain W4 稍稍好一点


#### 1.5.5 文献
- Lin et al., *AWQ: Activation-aware Weight Quantization for LLM Compression and Acceleration*, arXiv:2306.00978  
- 官方实现：https://github.com/mit-han-lab/llm-awq

## 1.6 INT4 / INT3 / Hybrid Quantization

- 为什么大模型常用 **权重低比特（INT4/INT3）+ 激活高比特（FP16/BF16）**  
- QLoRA 中的 NF4、Block Quantization、Double Quantization 数学形式  
- 简单的 PyTorch NF4 按块量化实现示例  

---

### 1.6.1 Hybrid Quantization 概念

Hybrid Quantization = 低比特权重 + 高精度激活：

- 权重：INT4 / INT3（weight-only quantization）
- 激活：FP16 / BF16
- 敏感层：使用 FP16 或 INT8

原因：

1. 权重是静态张量，只需量化一次，误差不会随时间累积。  
2. 激活动态范围大，量化会导致数值不稳定（如 LayerNorm、Softmax、GELU）。  
3. 在车端 GPU/NPU 上，减少权重带宽是提升吞吐/降低功耗的关键。

---

### 1.6.2. QLoRA 的三大贡献（数学视角）

####  NF4：NormalFloat 4-bit codebook

QLoRA 使用 **NormalFloat 4-bit（NF4）**，用一组非均匀量化点近似高斯分布（而非等间距整数）。

定义一个 4-bit codebook：

$$
\mathcal{Q}_{\text{NF4}} = \{q_0, q_1, \dots, q_{15}\}
$$

实际可以近似为：

$$
[-1.0,\,-0.66,\,-0.45,\,-0.32,\,-0.21,\,-0.14,\,-0.08,\,-0.03,\,
\;0.03,\;0.08,\;0.14,\;0.21,\;0.32,\;0.45,\;0.66,\;1.0]
$$

对一个权重标量 $w$，先按缩放因子 $s$ 归一化再量化：

$$
w_{\text{norm}} = \frac{w}{s}, \quad
\hat{q} = \arg\min_{q \in \mathcal{Q}_{\text{NF4}}} \lvert w_{\text{norm}} - q \rvert,
$$

反量化：

$$
\hat{w} = s \cdot \hat{q}.
$$

---

#### Block Quantization（按块量化）

设权重矩阵：

$$
W \in \mathbb{R}^{d_{\text{out}} \times d_{\text{in}}}
$$

按行切分为多个 block：

$$
W =
\begin{bmatrix}
W^{(1)} \\
W^{(2)} \\
\vdots \\
W^{(B)}
\end{bmatrix},
\quad
W^{(k)} \in \mathbb{R}^{b \times d_{\text{in}}},
$$

其中 $b$ 是 block size（如 32 或 64）。

对第 $k$ 个 block：

1. 计算缩放因子（例如取最大绝对值）：

$$
s_k = \max_{i,j} \lvert W^{(k)}_{ij} \rvert + \varepsilon
$$

2. 归一化并映射到 NF4 codebook：

$$
\hat{W}^{(k)} = s_k \cdot Q_{\text{NF4}}\!\left(\frac{W^{(k)}}{s_k}\right),
$$

其中 $Q_{\text{NF4}}$ 表示逐元素选择最近的 NF4 量化点。

整体可以写成：

$$
\hat{W} =
\begin{bmatrix}
\hat{W}^{(1)} \\
\hat{W}^{(2)} \\
\vdots \\
\hat{W}^{(B)}
\end{bmatrix}.
$$

---

#### Double Quantization（两次量化）

第一次量化（权重）：

$$
W \rightarrow (q_i, s_k)
$$

含义：

- 原始权重 $W$ 用 4-bit NF4 索引 $q_i$ 表示  
- 每个 block 有一个 FP16/FP32 的缩放因子 $s_k$

对于大模型，所有 $s_k$ 的存储开销仍然显著。  
Double Quantization 的想法是：**连缩放因子本身也量化**。

第二次量化（对 $s_k$ 再量化成低比特，如 INT8）：

$$
s_k \rightarrow s_{k,q}.
$$

举个简单形式：

- 选一个全局 scale $\alpha$，把所有 $s_k$ 归一化  
- 存储整数 $z_k$：

$$
z_k = \text{round}\!\left(\frac{s_k}{\alpha}\right), \quad s_k \approx \alpha \cdot z_k.
$$

于是最终存储内容为：

- 权重索引：$q_i$（4-bit）  
- 缩放因子索引：$z_k$（8-bit）  

这样可以把整体模型的有效比特数更接近 “真正的 4-bit”，而不是 “4-bit 权重 + 大量 16-bit scale”。

---

### 1.6.3. 为什么 INT4 / INT3 仍然可以保持高精度？

考虑一层线性层（忽略 bias）：

$$
Y = X W,
$$

其中 $X \in \mathbb{R}^{n \times d_{\text{in}}}$，$W \in \mathbb{R}^{d_{\text{in}} \times d_{\text{out}}}$。

量化后的权重为：

$$
\hat{W} = W + \Delta W.
$$

对应输出为：

$$
\hat{Y} = X \hat{W} = X (W + \Delta W) = XW + X \Delta W.
$$

误差为：

$$
\Delta Y = \hat{Y} - Y = X \Delta W.
$$

只要满足：

- $\lVert \Delta W \rVert$ 足够小（NF4 的量化误差相对传统 INT4 更小）；  
- 激活 $X$ 使用 FP16 / BF16，不再引入额外的低比特误差；  

则：

$$
\frac{\lVert \Delta Y \rVert}{\lVert Y \rVert}
$$

可以被控制在较小范围，即模型精度基本不掉。

QLoRA 的实验表明，在 LLaMA-65B 这类大模型上，**INT4 权重 + FP16 激活** 的精度几乎可以匹配原始 FP16（损失 < 1%）。

---

### 1.6.4. 工程落地（车端 / 边缘设备）

在典型 Transformer 结构中，可采取以下混合精度配置：

| 层类型 | 推荐精度 | 说明 |
|-------|----------|------|
| Q/K/V projection | INT4 | 权重接近高斯分布，冗余度高 |
| FFN up / down projection | INT4 或 INT3 | 参数量大，压缩收益高 |
| Output projection | INT4 | 容错性较高 |
| Embedding | INT8 / FP16 | lookup + 不规则访问，不适合 4bit |
| LayerNorm | FP16 | 对数值稳定高度敏感 |
| Softmax / attention score | FP16 | 涉及 $\\exp$ 和归一化，需高精度 |

部署策略示例：

1. **先做全模型 weight-only INT4（NF4）量化**。  
2. 通过校准数据，测 per-layer 误差（如输出 $L_2$ 相对误差），画 heatmap。  
3. 对误差最大的若干层回退为 INT8 或 FP16（Hybrid Quantization）。  
4. 尝试不同 block size（32 / 64 / 128），在精度与性能间寻找平衡点。

---

### 1.6.5 文献

- Dettmers et al., *QLoRA: Efficient Finetuning of Quantized LLMs*, arXiv:2305.14314
- 论文实现 https://github.com/artidoro/qlora

repo 里包含：

- 支持 4-bit NormalFloat (NF4) quantization

- “Double Quantization”（也就是不仅 quantize 权重，还 quantize quantization constants／scales）

- 与主流库集成，例如：用 bitsandbytes + transformers + PEFT 联合微调 / 推理流程 

- 脚本、示例、以及训练／推理 pipeline

⚠️ 注意事项
- 这个官方 repo 偏向 “finetuning + LoRA + quantized checkpoint” 场景 —— 如果你只是想做自定义量化 / 推理优化 / 或者底层部署（比如 INT4 + activation FP16 + custom kernel），可能需要修改或扩展 repo。

- 量化细节（block-size, double quant, scale 的管理等）是由 bitsandbytes + QLoRA 实现，但你如果想用其他 quantization 格式（比如你之前提到的 INT3 / hybrid quantization / 混合层精度） —— 可能要基于这个 repo 做一些工程级改造。


# 1.7 总结
| 方法 | PTQ（训练后量化） | Training-Aware / QAT | Deployment-Time（部署可独立做） | 说明 |
|------|-------------------|------------------------|----------------------------------|------|
| MinMax / Percentile / Basic INT8 | ✅ 是 | ❌ 否 | ✅ 是 | 最基础的激活/权重量化，只需校准数据 |
| KL Divergence / Histogram Calibration | ✅ 是 | ❌ 否 | ✅ 是 | 选激活 clipping threshold，部署常用 |
| GPTQ (Hessian-based PTQ) | ✅ 是 | ❌ 否 | ✅ 是 | 最强 PTQ 权重量化，只需校准输入 |
| AWQ (Activation-aware Weight Quant) | ✅ 是 | ❌ 否 | ✅ 是 | 基于 activation 的 PTQ，不需训练 |
| SmoothQuant | ⚠️ PTQ-ish | ❌ 否 | ✅ 是 | 训练后一次性 scaling，部署可做 |
| INT4/INT3/NF4 weight-only | ✅ 是 | ❌ 否 | ✅ 是 | 纯权重低比特量化，部署可处理 |
| NF4 + LoRA fine-tuning | ⚠️ 部分 | ✅ 是 | ❌ 否 | 量化本身是 PTQ，但 LoRA 需要训练 |
